In [ ]:
from sportspose.dataset import SportsPoseDataset
import matplotlib.pyplot as plt

jointspath = r"D:\SportsPose"
dataset = SportsPoseDataset(
    data_dir=jointspath,
    sample_level="frame",
)

sample = dataset[0]
frameFO = sample["video"]["image"]["FO"][0]
jointsFO = sample["joints_2d"]["FO"][0]

plt.figure()
plt.imshow(frameFO)
plt.scatter(jointsFO[:, 0], jointsFO[:, 1])
plt.show(block=True)